In [ ]:
"""
Schema Change Report Generator
Jupyter Notebook for analyzing PostgreSQL schema changes between two timestamps
"""
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [19]:
# pip install psycopg2-binary pandas matplotlib seaborn
# pip install openpyxl

In [20]:
# ============================================
# DATABASE CONNECTION CONFIGURATION
# ============================================

DB_CONFIG = {
    'host': 'localhost',
    'database': 'test_db',
    'user': 'manoj_anumalla',
    'password': '',
    'port': 5432
}

In [21]:
# START_TIME = '2025-11-12 18:12:50'
# END_TIME = '2025-11-12 18:25:50'

# START_TIME = '2025-11-12 18:13:39'
# END_TIME = '2025-11-12 18:18:55'
START_TIME = '2025-11-18 18:14:29'
END_TIME = '2025-11-18 21:36:37'



In [22]:
def get_connection():
    """Create database connection"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("Database connection successful")
        return conn
    except Exception as e:
        print(f"Database connection failed: {e}")
        return None

conn = get_connection()

Database connection successful


In [23]:
# ============================================
# 1. EXECUTIVE SUMMARY
# ============================================

print("="*80)
print("SCHEMA CHANGE REPORT")
print(f"Period: {START_TIME} to {END_TIME}")
print("="*80)

query_executive_summary = """
SELECT 
    COUNT(DISTINCT snapshot_id) as total_snapshots,
    COUNT(DISTINCT schema_name) as schemas_affected,
    COUNT(DISTINCT object_type_name) as tables_affected,
    COUNT(DISTINCT object_subtype_name) as columns_affected,
    COUNT(*) as total_changes,
    MIN(processed_time) as first_change,
    MAX(processed_time) as last_change
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s;
"""

df_summary = pd.read_sql_query(query_executive_summary, conn, params=(START_TIME, END_TIME))

print("\n EXECUTIVE SUMMARY")
print("-" * 80)
for col in df_summary.columns:
    print(f"{col.replace('_', ' ').title()}: {df_summary[col].iloc[0]}")


SCHEMA CHANGE REPORT
Period: 2025-11-18 18:14:29 to 2025-11-18 21:36:37

 EXECUTIVE SUMMARY
--------------------------------------------------------------------------------
Total Snapshots: 24
Schemas Affected: 1
Tables Affected: 2
Columns Affected: 30
Total Changes: 70
First Change: 2025-11-18 18:14:29.872481
Last Change: 2025-11-18 21:32:51.652349


In [24]:
# ============================================
# 2. CHANGE TYPE BREAKDOWN
# ============================================

# query_change_breakdown = """
# SELECT 
#     change_type,
#     COUNT(*) as count,
#     ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
# FROM pdcd_schema.md5_metadata_tbl
# WHERE processed_time BETWEEN %s AND %s
# GROUP BY change_type
# ORDER BY count DESC;
# """
query_change_breakdown = """
SELECT 
    'CHANGE TYPE BREAKDOWN' as report_section,
    change_type,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
GROUP BY change_type
ORDER BY count DESC;
"""
df_change_types = pd.read_sql_query(query_change_breakdown, conn, params=(START_TIME, END_TIME))

print("\n CHANGE TYPE BREAKDOWN")
print("-" * 80)
print(df_change_types.to_string(index=False))



 CHANGE TYPE BREAKDOWN
--------------------------------------------------------------------------------
       report_section change_type  count  percentage
CHANGE TYPE BREAKDOWN       ADDED     36       51.43
CHANGE TYPE BREAKDOWN    MODIFIED     22       31.43
CHANGE TYPE BREAKDOWN     RENAMED      7       10.00
CHANGE TYPE BREAKDOWN     DELETED      5        7.14


In [25]:
# ============================================
# 3. CHANGES BY TABLE
# ============================================

query_changes_by_table = """
SELECT 
    schema_name,
    object_type_name as table_name,
    COUNT(*) as total_changes,
    COUNT(*) FILTER (WHERE change_type = 'ADDED') as added,
    COUNT(*) FILTER (WHERE change_type = 'MODIFIED') as modified,
    COUNT(*) FILTER (WHERE change_type = 'DELETED') as deleted,
    COUNT(*) FILTER (WHERE change_type = 'RENAMED') as renamed
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
GROUP BY schema_name, object_type_name
ORDER BY total_changes DESC;
"""

df_by_table = pd.read_sql_query(query_changes_by_table, conn, params=(START_TIME, END_TIME))

print("\n CHANGES BY TABLE")
print("-" * 80)
print(df_by_table.to_string(index=False))



 CHANGES BY TABLE
--------------------------------------------------------------------------------
     schema_name  table_name  total_changes  added  modified  deleted  renamed
analytics_schema   employees             41     22        15        1        3
analytics_schema departments             29     14         7        4        4


In [26]:
# ============================================
# 4. DETAILED COLUMN ADDITIONS
# ============================================

query_additions = """
SELECT 
    snapshot_id,
    processed_time,
    schema_name,
    object_type_name as table_name,
    object_subtype_name as column_name,
    REGEXP_REPLACE(object_subtype_details, '.*data_type:([^,]+).*', '\\1') as data_type,
    REGEXP_REPLACE(object_subtype_details, '.*max_length:([^,]*),.*', '\\1') as max_length,
    REGEXP_REPLACE(object_subtype_details, '.*nullable:([^,]+).*', '\\1') as nullable,
    REGEXP_REPLACE(object_subtype_details, '.*default_value:([^,]*),.*', '\\1') as default_value,
    REGEXP_REPLACE(object_subtype_details, '.*ordinal_position:([0-9]+).*', '\\1') as position
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
    AND change_type = 'ADDED'
ORDER BY processed_time, schema_name, object_type_name, object_subtype_name;
"""

df_additions = pd.read_sql_query(query_additions, conn, params=(START_TIME, END_TIME))

print("\n COLUMN ADDITIONS")
print("-" * 80)
if df_additions.empty:
    print("No columns added in this period.")
else:
    print(df_additions.to_string(index=False))


 COLUMN ADDITIONS
--------------------------------------------------------------------------------
 snapshot_id             processed_time      schema_name  table_name                  column_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       data_type                                                                                                                                                                   

In [27]:
# ============================================
# 5. DETAILED COLUMN MODIFICATIONS
# ============================================

query_modifications = """
WITH current_changes AS (
    SELECT 
        snapshot_id,
        processed_time,
        schema_name,
        object_type_name,
        object_subtype_name,
        object_subtype_details,
        object_md5
    FROM pdcd_schema.md5_metadata_tbl
    WHERE processed_time BETWEEN %s AND %s
        AND change_type = 'MODIFIED'
),
previous_state AS (
    SELECT DISTINCT ON (cc.schema_name, cc.object_type_name, cc.object_subtype_name)
        cc.schema_name,
        cc.object_type_name,
        cc.object_subtype_name,
        m.object_subtype_details as old_details,
        m.processed_time as old_time
    FROM current_changes cc
    JOIN pdcd_schema.md5_metadata_tbl m 
        ON cc.schema_name = m.schema_name 
        AND cc.object_type_name = m.object_type_name
        AND cc.object_subtype_name = m.object_subtype_name
        AND m.processed_time < cc.processed_time
    ORDER BY cc.schema_name, cc.object_type_name, cc.object_subtype_name, m.processed_time DESC
)
SELECT 
    cc.snapshot_id,
    cc.processed_time,
    cc.schema_name,
    cc.object_type_name as table_name,
    cc.object_subtype_name as column_name,
    REGEXP_REPLACE(ps.old_details, '.*data_type:([^,]+).*', '\\1') as old_data_type,
    REGEXP_REPLACE(cc.object_subtype_details, '.*data_type:([^,]+).*', '\\1') as new_data_type,
    REGEXP_REPLACE(ps.old_details, '.*max_length:([^,]*),.*', '\\1') as old_max_length,
    REGEXP_REPLACE(cc.object_subtype_details, '.*max_length:([^,]*),.*', '\\1') as new_max_length,
    REGEXP_REPLACE(ps.old_details, '.*nullable:([^,]+).*', '\\1') as old_nullable,
    REGEXP_REPLACE(cc.object_subtype_details, '.*nullable:([^,]+).*', '\\1') as new_nullable
FROM current_changes cc
LEFT JOIN previous_state ps 
    ON cc.schema_name = ps.schema_name 
    AND cc.object_type_name = ps.object_type_name
    AND cc.object_subtype_name = ps.object_subtype_name
ORDER BY cc.processed_time, cc.schema_name, cc.object_type_name;
"""

df_modifications = pd.read_sql_query(query_modifications, conn, params=(START_TIME, END_TIME))

print("\n COLUMN MODIFICATIONS")
print("-" * 80)
if df_modifications.empty:
    print("No columns modified in this period.")
else:
    # Create readable change descriptions
    changes = []
    for _, row in df_modifications.iterrows():
        change_desc = []
        if row['old_data_type'] != row['new_data_type']:
            change_desc.append(f"Type: {row['old_data_type']} → {row['new_data_type']}")
        if row['old_max_length'] != row['new_max_length']:
            change_desc.append(f"Length: {row['old_max_length']} → {row['new_max_length']}")
        if row['old_nullable'] != row['new_nullable']:
            change_desc.append(f"Nullable: {row['old_nullable']} → {row['new_nullable']}")
        changes.append('; '.join(change_desc) if change_desc else 'Other changes')
    
    df_modifications['changes'] = changes
    print(df_modifications[['processed_time', 'table_name', 'column_name', 'changes']].to_string(index=False))



 COLUMN MODIFICATIONS
--------------------------------------------------------------------------------
            processed_time  table_name                 column_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [28]:
# ============================================
# 6. COLUMN DELETIONS
# ============================================

query_deletions = """
SELECT 
    snapshot_id,
    processed_time,
    schema_name,
    object_type_name as table_name,
    object_subtype_name as column_name,
    REGEXP_REPLACE(object_subtype_details, '.*data_type:([^,]+).*', '\\1') as data_type,
    REGEXP_REPLACE(object_subtype_details, '.*ordinal_position:([0-9]+).*', '\\1') as position
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
    AND change_type = 'DELETED'
ORDER BY processed_time, schema_name, object_type_name;
"""

df_deletions = pd.read_sql_query(query_deletions, conn, params=(START_TIME, END_TIME))

print("\n COLUMN DELETIONS")
print("-" * 80)
if df_deletions.empty:
    print("No columns deleted in this period.")
else:
    print(df_deletions.to_string(index=False))


 COLUMN DELETIONS
--------------------------------------------------------------------------------
 snapshot_id             processed_time      schema_name  table_name                 column_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      data_type                                                                                                                                                                     

In [29]:
# ============================================
# 7. COLUMN RENAMES -- //Can be optimized to show old and new names
# ============================================

query_renames = """
SELECT 
    snapshot_id,
    processed_time,
    schema_name,
    object_type_name as table_name,
    object_subtype_name as new_name,
    object_md5
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
    AND change_type = 'RENAMED'
ORDER BY processed_time, schema_name, object_type_name;
"""

df_renames = pd.read_sql_query(query_renames, conn, params=(START_TIME, END_TIME))

print("\n COLUMN RENAMES")
print("-" * 80)
if df_renames.empty:
    print("No columns renamed in this period.")
else:
    print(df_renames.to_string(index=False))


 COLUMN RENAMES
--------------------------------------------------------------------------------
 snapshot_id             processed_time      schema_name  table_name                     new_name                       object_md5
           3 2025-11-18 18:24:24.633620 analytics_schema   employees      trg_emp_insert_audit_v2 82e54f0c8b98900e9ad56097a80ebd4c
           5 2025-11-18 18:42:18.597521 analytics_schema   employees            trg_check_salary1 e5fdfaa3559c42448fa2ad8f8705bb58
          11 2025-11-18 20:00:53.811601 analytics_schema departments  trg_department_update_audit c2b7bfc45e5cb485cc9c7a23e6b6b4a1
          12 2025-11-18 20:02:31.397699 analytics_schema departments  trg_department_insert_audit c2b7bfc45e5cb485cc9c7a23e6b6b4a1
          21 2025-11-18 20:43:42.923025 analytics_schema departments trg_department_update_audit1 b7f10e8b788086eaa9dd079555204c13
          22 2025-11-18 20:47:05.071896 analytics_schema departments  trg_department_update_audit b7f10e8b788086eaa9

In [30]:
# ============================================
# 8. CHRONOLOGICAL TIMELINE
# ============================================

query_timeline = """
SELECT 
    snapshot_id,
    processed_time,
    schema_name,
    object_type_name as table_name,
    object_subtype_name as column_name,
    change_type
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
ORDER BY processed_time, snapshot_id, schema_name, object_type_name;
"""

df_timeline = pd.read_sql_query(query_timeline, conn, params=(START_TIME, END_TIME))

print("\n CHRONOLOGICAL TIMELINE")
print("-" * 80)
if df_timeline.empty:
    print("No changes in this period.")
else:
    # Add emoji indicators
    emoji_map = {
        'ADDED': '',
        'MODIFIED': '',
        'DELETED': '',
        'RENAMED': ''
    }
    df_timeline['change_indicator'] = df_timeline['change_type'].map(emoji_map)
    df_timeline['description'] = (df_timeline['change_indicator'] + ' ' + 
                                  df_timeline['change_type'] + ': ' +
                                  df_timeline['table_name'] + '.' + 
                                  df_timeline['column_name'])
    print(df_timeline[['processed_time', 'snapshot_id', 'description']].to_string(index=False))

    


 CHRONOLOGICAL TIMELINE
--------------------------------------------------------------------------------
            processed_time  snapshot_id                                        description
2025-11-18 18:14:29.872481            1                   ADDED: departments.department_id
2025-11-18 18:14:29.875372            1                 ADDED: departments.department_name
2025-11-18 18:14:29.875383            1                   ADDED: departments.main_location
2025-11-18 18:14:29.875390            1                ADDED: departments.ternary_location
2025-11-18 18:14:29.875397            1                      ADDED: departments.manager_id
2025-11-18 18:14:29.875404            1                     ADDED: departments.budget_code
2025-11-18 18:14:29.875409            1                       ADDED: employees.employee_id
2025-11-18 18:14:29.875414            1                        ADDED: employees.first_name
2025-11-18 18:14:29.875420            1                         ADDED: empl

In [31]:
# ============================================
# 9. HIGH-RISK CHANGES
# ============================================

query_high_risk = """
SELECT 
    processed_time,
    schema_name,
    object_type_name as table_name,
    object_subtype_name as column_name,
    change_type,
    CASE 
        WHEN change_type = 'DELETED' THEN ' CRITICAL: Column deleted - potential data loss'
        WHEN change_type = 'MODIFIED' THEN ' MEDIUM: Data type or constraint changed'
        WHEN change_type = 'RENAMED' THEN ' MEDIUM: Column renamed - update application code'
        ELSE ' LOW: Standard change'
    END as risk_level
FROM pdcd_schema.md5_metadata_tbl
WHERE processed_time BETWEEN %s AND %s
    AND change_type IN ('DELETED', 'MODIFIED', 'RENAMED')
ORDER BY 
    CASE 
        WHEN change_type = 'DELETED' THEN 1
        WHEN change_type = 'MODIFIED' THEN 2
        ELSE 3
    END,
    processed_time;
"""

df_high_risk = pd.read_sql_query(query_high_risk, conn, params=(START_TIME, END_TIME))

print("\n HIGH-RISK CHANGES")
print("-" * 80)
if df_high_risk.empty:
    print("No high-risk changes detected.")
else:
    print(df_high_risk.to_string(index=False))


 HIGH-RISK CHANGES
--------------------------------------------------------------------------------
            processed_time      schema_name  table_name                  column_name change_type                                        risk_level
2025-11-18 20:03:11.482455 analytics_schema departments  trg_department_update_audit     DELETED    CRITICAL: Column deleted - potential data loss
2025-11-18 20:24:39.914327 analytics_schema departments  trg_department_update_audit     DELETED    CRITICAL: Column deleted - potential data loss
2025-11-18 20:35:25.231972 analytics_schema departments  trg_department_update_audit     DELETED    CRITICAL: Column deleted - potential data loss
2025-11-18 20:50:03.602577 analytics_schema departments  trg_department_update_audit     DELETED    CRITICAL: Column deleted - potential data loss
2025-11-18 21:27:48.455763 analytics_schema   employees  trg_employee_delete_cleanup     DELETED    CRITICAL: Column deleted - potential data loss
2025-11-18 18:16:

In [32]:
# ============================================
# EXPORT OPTIONS
# ============================================

print("\n" + "="*80)
print("EXPORT OPTIONS")
print("="*80)

# Export to Excel
try:
    with pd.ExcelWriter(f'schema_change_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx') as writer:
        df_summary.to_excel(writer, sheet_name='Summary', index=False)
        df_change_types.to_excel(writer, sheet_name='Change Types', index=False)
        df_by_table.to_excel(writer, sheet_name='By Table', index=False)
        df_additions.to_excel(writer, sheet_name='Additions', index=False)
        df_modifications.to_excel(writer, sheet_name='Modifications', index=False)
        df_deletions.to_excel(writer, sheet_name='Deletions', index=False)
        df_renames.to_excel(writer, sheet_name='Renames', index=False)
        df_timeline.to_excel(writer, sheet_name='Timeline', index=False)
        df_high_risk.to_excel(writer, sheet_name='High Risk', index=False)
    print(" Excel report exported successfully")
except Exception as e:
    print(f" Excel export failed: {e}")

# Export to CSV
try:
    df_timeline.to_csv(f'schema_changes_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv', index=False)
    print(" CSV export successful")
except Exception as e:
    print(f" CSV export failed: {e}")

# Close connection
if conn:
    conn.close()
    print("\n Database connection closed")

print("\n" + "="*80)
print("REPORT GENERATION COMPLETE")
print("="*80)


EXPORT OPTIONS
 Excel export failed: No module named 'openpyxl'
 CSV export successful

 Database connection closed

REPORT GENERATION COMPLETE
